### Library Import

In [1]:
import os
from typing import Any, List, Dict
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
import optuna
from data_preprocessing import *
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


### Data Load

In [2]:
# 파일 호출
data_path: str = "/data/ephemeral/home/BTC/data"
train_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv")).assign(_type="train") # train 에는 _type = train 
test_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")).assign(_type="test") # test 에는 _type = test
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
df: pd.DataFrame = pd.concat([train_df, test_df], axis=0)

In [3]:
# HOURLY_ 로 시작하는 .csv 파일 이름을 file_names 에 할딩
file_names: List[str] = [
    f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
]

# 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
    _rename_rule = {
        col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")


100%|██████████| 107/107 [00:03<00:00, 31.29it/s]


### EDA (Explanatory Data Analysis)

### Feature engineering

In [4]:
# 모델에 사용할 컬럼, 컬럼의 rename rule을 미리 할당함
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type" : "_type",
    "hourly_market-data_open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_close": "close",
    "hourly_network-data_difficulty_difficulty": "difficulty",
    "hourly_network-data_supply_supply_total": "supply_total",
    "hourly_network-data_utxo-count_utxo_count": "utxo_count"
}
df = df[cols_dict.keys()].rename(cols_dict, axis=1)
df.shape

(11552, 8)

In [5]:
# continuous 열을 따로 할당해둠
conti_cols: List[str] = [
    "close",
    "open_interest",
    "difficulty",
    "supply_total",
    "utxo_count"
]

# # 최대 24시간의 shift 피쳐를 계산
# shift_list = shift_feature(
#     df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 24)]
# )

# # concat 하여 df 에 할당
# df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

In [6]:
df

,ID,target,_type,open_interest,close,difficulty,supply_total,utxo_count
0,2023-01-01 00:00:00,2.0,train,6.271344e+09,16536.747967,3.536407e+13,1.924871e+07,83308092.0
1,2023-01-01 01:00:00,1.0,train,6.288683e+09,16557.136536,3.536407e+13,1.924874e+07,83314883.0
2,2023-01-01 02:00:00,1.0,train,6.286796e+09,16548.149805,3.536407e+13,1.924879e+07,83314090.0
3,2023-01-01 03:00:00,1.0,train,6.284575e+09,16533.632875,3.536407e+13,1.924882e+07,83326258.0
4,2023-01-01 04:00:00,2.0,train,6.291582e+09,16524.712159,3.536407e+13,1.924886e+07,83339168.0
...,...,...,...,...,...,...,...,...
11547,2024-04-26 03:00:00,NaN,test,1.486836e+10,NaN,8.810419e+13,NaN,179820708.0
11548,2024-04-26 04:00:00,NaN,test,NaN,NaN,8.810419e+13,NaN,179833897.0
11549,2024-04-26 05:00:00,NaN,test,NaN,NaN,8.810419e+13,NaN,179851249.0
11550,2024-04-26 06:00:00,NaN,test,NaN,NaN,8.810419e+13,NaN,179852452.0


In [7]:
# _type에 따라 train, test 분리
train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

### Model Training

xgboost 라이브러리에 구현되어 있는 XGBRegressor 모델을 사용하여 학습 및 평가를 진행합니다. xgboost의 래퍼 클래스(wrapper class) 중 **사이킷런 래퍼**를 사용할 예정입니다.

#### model parameter (XGBRegressor)
* n_estimator: 트리의 개수 (디폴트 = 100)  

* learning_rate: 학습 단계별 가중치를 얼마나 사용할지(이전 결과를 얼마나 반영할 것인지) 결정. 일반적으로 0.01 ~ 0.2

* max_depth: 트리의 최대 깊이. (디폴트 = 6) 일반적으로 3 ~ 10  

* min_child_weight: child에서 필요한 모든 관측치에 대한 가중치의 최소 합. 이 값보다 샘플 수가 작으면 leaf node가 된다. 너무 큰 값을 적용하면 과소적합이 될 수 있다.  

* early stopping_rounds: 최대한 몇 개의 트리를 완성해볼 것인지 결정. valid loss에 더 이상 진전이 없으면 멈춘다. n_estimator가 높을 때 주로 사용  

* gamma: 트리에서 추가적으로 가지를 나눌지를 결정할 최소 손실 감소값. 값이 클수록 과적합 감소 효과  

* subsample: 각 트리마다 데이터 샘플링 비율. (디폴트 = 1) 일반적으로 0.5 ~ 1  

* colsample_bytree: 각 트리마다 feature 샘플링 비율. (디폴트 = 1) 일반적으로 0.5 ~ 1  

* reg_lambda: L2 regularization 가중치 (디폴트 = 1)  

* reg_alpha: L1 regularization 가중치 (디폴트 = 1)  

* scale_pos_weight: 데이터가 불균형할때 사용, 0보다 큰 값. (디폴트 = 1) 보통 값을 (음성 데이터 수)/(양성 데이터 수) 값으로 한다. 

#### fit 파라미터

* early_stopping_rounds:
* eval_metric: 
* eval_set:


In [8]:
X_train = train_df.drop(["ID", "target", "close"], axis=1)
y_train = train_df["close"]
target = train_df["target"]

In [9]:
def close_to_class(series: pd.Series) -> pd.Series:
    """close 변수를 target값으로 변환하는 함수입니다.

    Args:
        series (pd.Series): 변환을 원하는 close 변수

    Returns:
        pd.Series: 변환된 target 값
    """
    close = pd.DataFrame()
    close['close'] = series
    close['close_lag1'] = close['close'].shift(1)
    close['close_lag1_percent'] = (close['close'] - close['close_lag1']) / close['close_lag1']
    close['class'] = close['close']
    for i in range(close.shape[0]):
        if close.loc[i, 'close_lag1_percent'] < -0.005:
            close.loc[i, 'class'] = 0
        elif close.loc[i, 'close_lag1_percent'] < 0:
            close.loc[i, 'class'] = 1
        elif close.loc[i, 'close_lag1_percent'] < 0.005:
            close.loc[i, 'class'] = 2
        else:
            close.loc[i, 'class'] = 3
            
    return close["class"].shift(-1).fillna(method="ffill")

In [10]:
# 모델 평가
def evaluate(valid_target: pd.Series, 
             y_valid: pd.Series, 
             y_pred: np.ndarray, 
             metric: str
) -> float:
    """평가지표 metric을 반환하는 함수입니다.

    Args:
        valid_target: (pd.Series): 
        y_valid (pd.Series): 
        y_pred (np.ndarray): 모델을 사용하여 예측한 변수
        metric (str): 사용할 평가지표 metric 이름

    Returns:
        float: 사용할 평가지표 metric 값
    """
    if metric == "accuracy":
        classes = close_to_class(y_pred)
        return accuracy_score(valid_target, classes)
    elif metric == "mae":
        return mean_absolute_error(y_valid, y_pred)
    elif metric == "mse":
        return mean_squared_error(y_valid, y_pred)
    elif metric == "mape":
        return mean_absolute_percentage_error(y_valid, y_pred)

In [11]:
def model_train(model: Any, 
                X_train: pd.DataFrame, 
                y_train: pd.Series, 
                cv: int, 
                metric: str, 
) -> float:
    """K-Fold로 데이터를 분할한 후 전처리를 거쳐 주어진 모델로 데이터를 학습 및 평가를 진행합니다.

    Args:
        model (Any): 사용하는 모델 객체
        X_train (pd.DataFrame): 설명변수로 이루어진 학습 데이터프레임
        y_train (pd.Seris): 예측변수로 이루어진 학습 시리즈
        cv (int): 교차검증시 분할할 폴드의 수
        metric (str): 사용할 평가지표 metric 이름

    Returns:
        Any, float: 
    """
    kfold = KFold(n_splits=cv)
    # kfold = KFold(n_splits=cv, shuffle=True, random_state=42)  # shuffle 켰을 때
    score_list = []
    
    # warm_start는 모델의 속성으로, 같은 모델을 반복 학습할 때 이전 학습에서 학습된 파라미터를 초기화하지 않고 이어서 학습을 진행하는 옵션
    if hasattr(model, "warm_start"):
        model.warm_start = True

    # K-Fold 교차 검증
    for train_index, valid_index in kfold.split(X_train):
        X_train_fold, y_train_fold = X_train.iloc[train_index], y_train.iloc[train_index]
        X_valid, y_valid = X_train.iloc[valid_index], y_train.iloc[valid_index]

        valid_target = target[valid_index]
        
        # 전처리
        X_train_fold.fillna(X_train_fold.mean(), inplace=True)
        y_train_fold.fillna(y_train_fold.mean(), inplace=True)
        X_valid.fillna(X_valid.mean(), inplace=True)
        y_valid.fillna(y_valid.mean(), inplace=True)  # 이 부분을 mice와 같은 방법으로 조정할 예정. feature selection 등도 여기에서.

        
        # 모델 학습
        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_valid)
        score = evaluate(valid_target, y_valid, y_pred, metric=metric)  # 평가지표 metric 반환
        score_list.append(score)
    
    return np.mean(score_list)

In [12]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.3, 1.0),
        "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
        "device": "gpu",
        "random_state": 42
    }
    
    xgb_model = XGBRegressor(**params)
    acc = model_train(xgb_model, X_train, y_train, cv=5, metric="accuracy")
    return acc

In [13]:
# Optuna study 생성 및 최적화 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters: ", study.best_params)

# 최적의 하이퍼파라미터를 사용하여 최종 모델 생성
best_params = study.best_params
best_params["device"] = "gpu"
best_params["random_state"] = 42
best_xgb_model = XGBRegressor(**best_params)

# 최종 모델 평가
acc = model_train(best_xgb_model, X_train, y_train, cv=5, metric="accuracy")
print(f"XGBoost model accuracy: {acc}")

[I 2024-09-24 21:17:39,908] A new study created in memory with name: no-name-1a90c5b5-b90c-455c-8664-e56a944cc66b
[I 2024-09-24 21:17:43,371] Trial 0 finished with value: 0.4265981735159817 and parameters: {'n_estimators': 250, 'learning_rate': 0.002200629863982853, 'max_depth': 4, 'min_child_weight': 1, 'colsample_bytree': 0.5495564512723209, 'subsample': 0.7955436533174249}. Best is trial 0 with value: 0.4265981735159817.
[I 2024-09-24 21:17:45,856] Trial 1 finished with value: 0.42271689497716897 and parameters: {'n_estimators': 103, 'learning_rate': 0.0014207344061369256, 'max_depth': 3, 'min_child_weight': 5, 'colsample_bytree': 0.6832856917627255, 'subsample': 0.9323033460453602}. Best is trial 0 with value: 0.4265981735159817.
[I 2024-09-24 21:17:49,161] Trial 2 finished with value: 0.41689497716894974 and parameters: {'n_estimators': 159, 'learning_rate': 0.06633601878187961, 'max_depth': 7, 'min_child_weight': 5, 'colsample_bytree': 0.8381230142214211, 'subsample': 0.606326239

Best Hyperparameters:  {'n_estimators': 285, 'learning_rate': 0.0024171439975190358, 'max_depth': 6, 'min_child_weight': 1, 'colsample_bytree': 0.31834256836624825, 'subsample': 0.702400056995305}
XGBoost model accuracy: 0.4449771689497717


### Inference

In [14]:
X_test = test_df.drop(["ID", "target", "close"], axis=1)
X_test.fillna(X_test.mean(), inplace=True)

In [15]:
y_test_pred = best_xgb_model.predict(X_test)
y_test_pred_class = close_to_class(y_test_pred)

### Output File Save

In [16]:
# output file 할당후 save 
submission_df = submission_df.assign(target = y_test_pred_class)
submission_df["target"] = submission_df["target"].astype(np.int8)
submission_df.to_csv("output.csv", index=False)

In [17]:
out = pd.read_csv("output.csv")
out["target"].unique()

array([2, 1])

In [19]:
prop2 = (out["target"] == 2).mean()
count2 = (out["target"] == 2).sum()
prop1 = (out["target"] == 1).mean()
count1 = (out["target"] == 1).sum()

print(f"proportion of 2: {prop2}")
print(f"# of 2: {count2}")
print(f"proportion of 2: {prop1}")
print(f"# of 1: {count1}")

proportion of 2: 0.9760028653295129
# of 2: 2725
proportion of 2: 0.023997134670487107
# of 1: 67
